In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import sklearn as sk
import os
import random
import math

In [2]:
import sys
sys.version

'3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]'

In [3]:
tf.test.is_built_with_cuda()

True

In [19]:
gpus = tf.config.list_physical_devices('GPU')

print(gpus)

print("Num Physical Devices Available: ", len(gpus))

print(tf.__version__)
print(cv2.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num Physical Devices Available:  1
2.8.0
4.6.0


In [20]:
if gpus:
      # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


**Constants**


In [15]:
IMG_SIZE = 224
BATCH_SIZE = 32
#PATH = "C:/Python/chen22/hitting_behaviors/video_data/violence_dataset/"
PATH = "video_data/violence_dataset/"

**Make Datatset**


In [6]:
def video_to_frames(p):
    #assert os.path.exists(p)
    cap = cv2.VideoCapture(p)
    
    #assert cap
    rate = math.floor(cap.get(3))

    ImageFrames = []
    #assert cap.isOpened()
    

    '''make sure to figure out how to extract a certain amount of frames so you don't use too much memory'''
    while cap.isOpened():
        #ID = cap.get(1)
        
        success, image = cap.read()
        n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(n_frames)
        #assert success

        if success:

            for frame in range(n_frames):
                if frame % 10 == 0:
                    
                    
                    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                    ImageFrames.append(image)
                else: continue
            cap.release()
    #print("successful frame capture")
            return ImageFrames

        else: 
            break

In [7]:
CATEGORIES = ["NonViolence", "Violence"]
X_set = []
y_set = []
for category in os.listdir(PATH):
    
    
    path = os.path.join(PATH, category)
    #print(path)
    #assert os.path.exists(path)


    class_num = CATEGORIES.index(category)
    for i, video in tqdm(enumerate(os.listdir(path)[0:350]), total=len(os.listdir(path)[0:350])):

        #print(path + '/' + video)
        #print(os.path.join(path, video))
        frames = video_to_frames(path + '/' + video)
        if frames:
        
            for j, frame in enumerate(frames):
                X_set.append(frame)
                y_set.append(class_num)
        else: continue

            
print("success")

100%|██████████| 350/350 [00:05<00:00, 60.96it/s]

success


In [8]:
X_set = np.array(X_set).reshape(-1 , IMG_SIZE * IMG_SIZE * 3)
y_original = np.array(y_set)
len(X_set), len(y_set)

(9878, 9878)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_set, y_set, test_size=0.2, random_state=42)

X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255



In [10]:
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [11]:
type(X_train), type(y_train)

(numpy.ndarray, numpy.ndarray)

**Building Model**

In [22]:
EffNet = tf.keras.applications.EfficientNetV2B0(include_top=True,
                                                weights=None,
                                                pooling='avg', 
                                                classes=1)



Compile model

In [23]:
EffNet.compile(loss="binary_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])

            

In [17]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Fit model

In [ ]:
tf.Confi

In [26]:
history = EffNet.fit(X_train,
                     y_train,
                     epochs=11,
                     batch_size=16,
                     verbose=1)
print("Training Complete.")

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.